<a href="https://colab.research.google.com/github/anandhc6/Assignment1/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb


In [15]:
#import the dataset 
from keras.datasets import fashion_mnist
#import the required libraries 
from matplotlib import pyplot
import numpy as np
#load the data into train and test 
(X_train,y_train),(X_test,y_test)=fashion_mnist.load_data()
#pyplot.imshow(X_train[7])


Activation function

In [16]:
def sigmoid(x):
	return 1.0 / (1.0 + exp(-x))
 
 # rectified linear function
def rectified(x):
	return max(0.0, x)
 
 # tanh activation function
def tanh(x):
	return (exp(x) - exp(-x)) / (exp(x) + exp(-x))

Feedforward

In [11]:
import numpy as np
d=5
n=10
o=5
W=np.empty(o*n)
len(W)
b=np.empty(n)